## Explaination for BLERSSI Location Prediction

## Clone Cisco Kubeflow Starter pack repository

In [1]:
BRANCH_NAME="dev" #Provide git branch name "master" or "dev"
! git clone -b $BRANCH_NAME https://github.com/CiscoAI/cisco-kubeflow-starter-pack.git

Cloning into 'cisco-kubeflow-starter-pack'...
remote: Enumerating objects: 479, done.
remote: Counting objects: 100% (479/479), done.
remote: Compressing objects: 100% (310/310), done.
remote: Total 4459 (delta 160), reused 347 (delta 95), pack-reused 3980
Receiving objects: 100% (4459/4459), 16.60 MiB | 6.03 MiB/s, done.
Resolving deltas: 100% (1665/1665), done.


In [2]:
! pip install pandas sklearn kfserving alibi dill --user

     |████████████████████████████████| 10.1 MB 6.5 MB/s eta 0:00:01
     |████████████████████████████████| 85 kB 3.1 MB/s  eta 0:00:01
     |████████████████████████████████| 217 kB 14.4 MB/s eta 0:00:01
     |████████████████████████████████| 177 kB 14.0 MB/s eta 0:00:01
     |████████████████████████████████| 6.8 MB 13.5 MB/s eta 0:00:01     |███████████████████▌            | 4.2 MB 13.5 MB/s eta 0:00:01
     |████████████████████████████████| 88 kB 8.3 MB/s  eta 0:00:01
     |████████████████████████████████| 75 kB 5.5 MB/s  eta 0:00:01
     |████████████████████████████████| 55 kB 5.1 MB/s  eta 0:00:01
     |████████████████████████████████| 10.0 MB 14.8 MB/s eta 0:00:01
     |████████████████████████████████| 115 kB 28.3 MB/s eta 0:00:01
     |████████████████████████████████| 273 kB 37.4 MB/s eta 0:00:01
     |████████████████████████████████| 48 kB 5.7 MB/s  eta 0:00:01
     |████████████████████████████████| 2.2 MB 39.5 MB/s eta 0:00:01
     |████████████████████████████████|

## Restart Notebook kernel

In [ ]:
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

## Import Libraries

In [1]:
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import pandas as pd
import numpy as np
import shutil
import yaml
import re
import os
import dill
import logging
import requests
import json
from sklearn.preprocessing import OneHotEncoder

from alibi.explainers import AnchorTabular
from kubernetes.client import V1Container
from kfserving import KFServingClient
from kfserving import constants
from kfserving import utils
from kfserving import V1alpha2EndpointSpec
from kfserving import V1alpha2PredictorSpec
from kfserving import V1alpha2InferenceServiceSpec
from kfserving import V1alpha2InferenceService
from kfserving import V1alpha2CustomSpec
from kfserving import V1alpha2Logger
from kfserving import V1alpha2TensorflowSpec
from kfserving import V1alpha2ExplainerSpec
from kfserving import V1alpha2AlibiExplainerSpec

from kubernetes import client as k8s_client
from kubernetes import config as k8s_config
from kubernetes.client.rest import ApiException

k8s_config.load_incluster_config()
api_client = k8s_client.CoreV1Api()
custom_api=k8s_client.CustomObjectsApi()
namespace = utils.get_default_target_namespace()
print(namespace)

anonymous


## Check GPUs availability

In [2]:
gpus = len(tf.config.experimental.list_physical_devices('GPU'))
if gpus == 0:
    print("Model will be trained using CPU")
elif gpus >= 0:
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
    tf.config.experimental.list_physical_devices('GPU')
    print("Model will be trained using GPU")

Model will be trained using CPU


## Declare Variables

In [3]:
path="cisco-kubeflow-starter-pack/apps/networking/ble-localization/onprem"
BLE_RSSI = pd.read_csv(os.path.join(path, "data/iBeacon_RSSI_Labeled.csv")) #Labeled dataset

# Configure model options
TF_DATA_DIR = os.getenv("TF_DATA_DIR", "/tmp/data/")
TF_MODEL_DIR = os.getenv("TF_MODEL_DIR", "blerssi/")
TF_EXPORT_DIR = os.getenv("TF_EXPORT_DIR", "blerssi/")
TF_MODEL_TYPE = os.getenv("TF_MODEL_TYPE", "DNN")
TF_TRAIN_STEPS = int(os.getenv("TF_TRAIN_STEPS", 5000))
TF_BATCH_SIZE = int(os.getenv("TF_BATCH_SIZE", 128))
TF_LEARNING_RATE = float(os.getenv("TF_LEARNING_RATE", 0.001))


# Feature columns
COLUMNS = list(BLE_RSSI.columns)
FEATURES = COLUMNS[2:]
def make_feature_cols():
  input_columns = [tf.feature_column.numeric_column(k) for k in FEATURES]
  return input_columns

## BLERSSI Input Dataset
### Attribute Information
location: The location of receiving RSSIs from ibeacons b3001 to b3013; 
          symbolic values showing the column and row of the location on the map (e.g., A01 stands for column A, row 1).
date: Datetime in the format of ‘d-m-yyyy hh:mm:ss’
b3001 - b3013: RSSI readings corresponding to the iBeacons; numeric, integers only.

In [4]:
BLE_RSSI.head(10)

,location,date,b3001,b3002,b3003,b3004,b3005,b3006,b3007,b3008,b3009,b3010,b3011,b3012,b3013
0,O02,10-18-2016 11:15:21,-200,-200,-200,-200,-200,-78,-200,-200,-200,-200,-200,-200,-200
1,P01,10-18-2016 11:15:19,-200,-200,-200,-200,-200,-78,-200,-200,-200,-200,-200,-200,-200
2,P01,10-18-2016 11:15:17,-200,-200,-200,-200,-200,-77,-200,-200,-200,-200,-200,-200,-200
3,P01,10-18-2016 11:15:15,-200,-200,-200,-200,-200,-77,-200,-200,-200,-200,-200,-200,-200
4,P01,10-18-2016 11:15:13,-200,-200,-200,-200,-200,-77,-200,-200,-200,-200,-200,-200,-200
5,P01,10-18-2016 11:15:11,-200,-200,-82,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200
6,P01,10-18-2016 11:15:09,-200,-200,-80,-200,-200,-77,-200,-200,-200,-200,-200,-200,-200
7,P02,10-18-2016 11:15:07,-200,-200,-86,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200
8,R01,10-18-2016 11:15:05,-200,-200,-200,-75,-200,-200,-200,-200,-200,-200,-200,-200,-200
9,R01,10-18-2016 11:15:03,-200,-200,-200,-75,-200,-200,-200,-200,-200,-200,-200,-200,-200


## Definition of Serving Input Receiver Function

In [5]:
feature_columns =  make_feature_cols()
inputs = {}
for feat in feature_columns:
  inputs[feat.name] = tf.placeholder(shape=[None], dtype=feat.dtype)
serving_input_receiver_fn = tf.estimator.export.build_raw_serving_input_receiver_fn(inputs)

## Train and Save BLE RSSI Model

In [6]:
# Feature columns
COLUMNS = list(BLE_RSSI.columns)
FEATURES = COLUMNS[2:]
LABEL = [COLUMNS[0]]

b3001 = tf.feature_column.numeric_column(key='b3001',dtype=tf.float64)
b3002 = tf.feature_column.numeric_column(key='b3002',dtype=tf.float64)
b3003 = tf.feature_column.numeric_column(key='b3003',dtype=tf.float64)
b3004 = tf.feature_column.numeric_column(key='b3004',dtype=tf.float64)
b3005 = tf.feature_column.numeric_column(key='b3005',dtype=tf.float64)
b3006 = tf.feature_column.numeric_column(key='b3006',dtype=tf.float64)
b3007 = tf.feature_column.numeric_column(key='b3007',dtype=tf.float64)
b3008 = tf.feature_column.numeric_column(key='b3008',dtype=tf.float64)
b3009 = tf.feature_column.numeric_column(key='b3009',dtype=tf.float64)
b3010 = tf.feature_column.numeric_column(key='b3010',dtype=tf.float64)
b3011 = tf.feature_column.numeric_column(key='b3011',dtype=tf.float64)
b3012 = tf.feature_column.numeric_column(key='b3012',dtype=tf.float64)
b3013 = tf.feature_column.numeric_column(key='b3013',dtype=tf.float64)
feature_columns = [b3001, b3002, b3003, b3004, b3005, b3006, b3007, b3008, b3009, b3010, b3011, b3012, b3013]

df_full = pd.read_csv(os.path.join(path, "data/iBeacon_RSSI_Labeled.csv")) #Labeled dataset

# Input Data Preprocessing 
df_full = df_full.drop(['date'],axis = 1)
df_full[FEATURES] = (df_full[FEATURES])/(-200)


#Output Data Preprocessing
dict = {'O02': 0,'P01': 1,'P02': 2,'R01': 3,'R02': 4,'S01': 5,'S02': 6,'T01': 7,'U02': 8,'U01': 9,'J03': 10,'K03': 11,'L03': 12,'M03': 13,'N03': 14,'O03': 15,'P03': 16,'Q03': 17,'R03': 18,'S03': 19,'T03': 20,'U03': 21,'U04': 22,'T04': 23,'S04': 24,'R04': 25,'Q04': 26,'P04': 27,'O04': 28,'N04': 29,'M04': 30,'L04': 31,'K04': 32,'J04': 33,'I04': 34,'I05': 35,'J05': 36,'K05': 37,'L05': 38,'M05': 39,'N05': 40,'O05': 41,'P05': 42,'Q05': 43,'R05': 44,'S05': 45,'T05': 46,'U05': 47,'S06': 48,'R06': 49,'Q06': 50,'P06': 51,'O06': 52,'N06': 53,'M06': 54,'L06': 55,'K06': 56,'J06': 57,'I06': 58,'F08': 59,'J02': 60,'J07': 61,'I07': 62,'I10': 63,'J10': 64,'D15': 65,'E15': 66,'G15': 67,'J15': 68,'L15': 69,'R15': 70,'T15': 71,'W15': 72,'I08': 73,'I03': 74,'J08': 75,'I01': 76,'I02': 77,'J01': 78,'K01': 79,'K02': 80,'L01': 81,'L02': 82,'M01': 83,'M02': 84,'N01': 85,'N02': 86,'O01': 87,'I09': 88,'D14': 89,'D13': 90,'K07': 91,'K08': 92,'N15': 93,'P15': 94,'I15': 95,'S15': 96,'U15': 97,'V15': 98,'S07': 99,'S08': 100,'L09': 101,'L08': 102,'Q02': 103,'Q01': 104}
df_full['location'] = df_full['location'].map(dict)
df_train=df_full.sample(frac=0.8,random_state=200)
df_valid=df_full.drop(df_train.index)

location_counts = BLE_RSSI.location.value_counts()
x1 = np.asarray(df_train[FEATURES])
y1 = np.asarray(df_train['location'])

x2 = np.asarray(df_valid[FEATURES])
y2 = np.asarray(df_valid['location'])

def formatFeatures(features):
    formattedFeatures = {}
    numColumns = features.shape[1]

    for i in range(0, numColumns):
        formattedFeatures["b"+str(3001+i)] = features[:, i]

    return formattedFeatures

trainingFeatures = formatFeatures(x1)
trainingCategories = y1

testFeatures = formatFeatures(x2)
testCategories = y2

# Train Input Function
def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((trainingFeatures, y1))
    dataset = dataset.repeat(1000).batch(TF_BATCH_SIZE)
    return dataset

# Test Input Function
def eval_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((testFeatures, y2))
    return dataset.repeat(1000).batch(TF_BATCH_SIZE)

# Provide list of GPUs should be used to train the model

distribution=tf.distribute.experimental.ParameterServerStrategy()
print('Number of devices: {}'.format(distribution.num_replicas_in_sync))

# Configuration of  training model

config = tf.estimator.RunConfig(train_distribute=distribution, model_dir=TF_MODEL_DIR, save_summary_steps=100, save_checkpoints_steps=100)

# Build 3 layer DNN classifier

model = tf.estimator.DNNClassifier(hidden_units = [13,65,110],
                 feature_columns = feature_columns,
                 model_dir = TF_MODEL_DIR,
                 n_classes=105, config=config
               )

export_final = tf.estimator.FinalExporter(TF_EXPORT_DIR, serving_input_receiver_fn=serving_input_receiver_fn)

train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, 
                                    max_steps=TF_TRAIN_STEPS)

eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn,
                                  steps=100,
                                  exporters=export_final,
                                  throttle_secs=1,
                                  start_delay_secs=1)

# Train and Evaluate the model

tf.estimator.train_and_evaluate(model, train_spec, eval_spec)

INFO:tensorflow:ParameterServerStrategy with compute_devices = ('/device:CPU:0',), variable_device = '/device:CPU:0'


[I 200708 11:30:20 parameter_server_strategy:294] ParameterServerStrategy with compute_devices = ('/device:CPU:0',), variable_device = '/device:CPU:0'


Number of devices: 1
INFO:tensorflow:Initializing RunConfig with distribution strategies.


[I 200708 11:30:20 run_config:566] Initializing RunConfig with distribution strategies.


INFO:tensorflow:Not using Distribute Coordinator.


...

INFO:tensorflow:SavedModel written to: blerssi/export/blerssi/temp-b'1594207909'/saved_model.pb


[I 200708 11:31:49 builder_impl:425] SavedModel written to: blerssi/export/blerssi/temp-b'1594207909'/saved_model.pb


INFO:tensorflow:Loss for final step: 261.33344.


[I 200708 11:31:49 estimator:371] Loss for final step: 261.33344.


({'accuracy': 0.26046875,
  'average_loss': 3.1123612,
  'loss': 398.38223,
  'global_step': 5000},
 [b'blerssi/export/blerssi/1594207909'])

## Define predict function

In [7]:
MODEL_EXPORT_PATH= os.path.join(TF_MODEL_DIR, "export", TF_EXPORT_DIR)

def predict(request):
    """ 
    Define custom predict function to be used by local prediction
    and explainer. Set anchor_tabular predict function so it always returns predicted class
    """
    # Get model exporter path
    for dir in os.listdir(MODEL_EXPORT_PATH):
        if re.match('[0-9]',dir):
            exported_path=os.path.join(MODEL_EXPORT_PATH,dir)
            break
    else:
        raise Exception("Model path not found")

    # Prepare model input data
    feature_cols=["b3001", "b3002","b3003","b3004","b3005","b3006","b3007","b3008","b3009","b3010","b3011","b3012","b3013"]
    input={'b3001': [], 'b3002': [], 'b3003': [], 'b3004': [], 'b3005': [], 'b3006': [], 'b3007': [], 'b3008': [], 'b3009': [], 'b3010': [], 'b3011': [], 'b3012': [], 'b3013': []}

    X=request
    if len(X)==13 and not type(X[0]) ==list:
        if type(X)==np.ndarray and len(X)==2:
            X=X.tolist()[-1]
        for i in range(len(X)):
            input[feature_cols[i]].append(X[i])
    else:
        for i in range(len(X)):
            for j in range(len(X[i])):
                input[feature_cols[j]].append(X[i][j])

    # Open a Session to predict
    with tf.Session() as sess:
        tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], exported_path)
        predictor= tf.contrib.predictor.from_saved_model(exported_path,signature_def_key='predict')
        output_dict= predictor(input)
    sess.close()
    class_ids=[]
    for id in output_dict["class_ids"]:
        class_ids.append(id[0])
    output={"predictions":np.array(class_ids).tolist()}
    return np.array(output["predictions"])

## Initialize and fit
To initialize the explainer, we provide a predict function, a list with the feature names to make the anchors easy to understand.

In [8]:
feature_cols=["b3001", "b3002", "b3003", "b3004", "b3005", "b3006", "b3007", "b3008", "b3009", "b3010", "b3011", "b3012", "b3013"]
explainer = AnchorTabular(predict, feature_cols)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.


[W 200708 11:31:56 deprecation:323] From <ipython-input-7-36ed7e487a7b>:33: load (from tensorflow.python.saved_model.loader_impl) is deprecated and will be removed in a future version.
    Instructions for updating:
    This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.


INFO:tensorflow:Restoring parameters from blerssi/export/blerssi/1594207909/variables/variables


[I 200708 11:31:56 saver:1284] Restoring parameters from blerssi/export/blerssi/1594207909/variables/variables


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



[W 200708 11:31:56 lazy_loader:50] 
    The TensorFlow contrib module will not be included in TensorFlow 2.0.
    For more information, please see:
      * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
      * https://github.com/tensorflow/addons
      * https://github.com/tensorflow/io (for I/O related ops)
    If you depend on functionality not listed there, please file an issue.
    


INFO:tensorflow:Restoring parameters from blerssi/export/blerssi/1594207909/variables/variables


[I 200708 11:31:56 saver:1284] Restoring parameters from blerssi/export/blerssi/1594207909/variables/variables


Discretize the ordinal features into quartiles. disc_perc is a list with percentiles used for binning

In [9]:
explainer.fit(x1, disc_perc=(25, 50, 75))

AnchorTabular(meta={
    'name': 'AnchorTabular',
    'type': ['blackbox'],
    'explanations': ['local'],
    'params': {'seed': None, 'disc_perc': (25, 50, 75)}
})

## Save Explainer file
Save explainer file with .dill extension. It will be used when creating the InferenceService

In [10]:
EXPLAINER_PATH="explainer"
if not os.path.exists(EXPLAINER_PATH):
    os.mkdir(EXPLAINER_PATH)
with open("%s/explainer.dill"%EXPLAINER_PATH, 'wb') as f:
    dill.dump(explainer,f)

## KFServing Client SDK

We will use the KFServing client SDK to create the InferenceService and deploy our custom predict image and explainer.

### Define InferenceService

Firstly define default endpoint spec, and then define the inferenceservice using the endpoint spec.

To use a custom image we need to use V1alphaCustomSpec which takes a V1Container from the kuberenetes library.

Kfserving custom image Source Code https://github.com/CiscoAI/cisco-kubeflow-starter-pack/tree/dev/apps/networking/ble-localization/onprem/explainer/model-server

In [11]:
MODEL_NAME="blerssi-model"
pvcname = !(echo  $HOSTNAME | sed 's/.\{2\}$//')
pvc = "workspace-"+pvcname[0]
api_version = constants.KFSERVING_GROUP + '/' + constants.KFSERVING_VERSION
default_endpoint_spec = V1alpha2EndpointSpec(
                          predictor=V1alpha2PredictorSpec(
                              custom=V1alpha2CustomSpec(
                                  container=V1Container(
                                      name="kfserving-container",
                                      image="docker.io/samba07/inference-service:0.1",
                                      env=[{"name":"STORAGE_URI", "value":"pvc://%s/%s"%(pvc,MODEL_EXPORT_PATH)}],
                                      command=["python"],
                                      args=[
                                          "model.py",
                                          "--storage_uri", "pvc://%s/%s"%(pvc,MODEL_EXPORT_PATH),
                                          "--out_dir", "/mnt/models/"
                                          ]))),
                           explainer=V1alpha2ExplainerSpec(
                           custom=V1alpha2CustomSpec(
                               container=V1Container(
                                   name="kfserving-container",
                                   image="gcr.io/kfserving/alibi-explainer@sha256:31d487ec6ec9abc6f851c4e481d75a77aea1d8e79d39f18b07bbc8684708fd3e",
                                   env=[{"name":"STORAGE_URI", "value":"pvc://%s/%s"%(pvc,EXPLAINER_PATH)}],
                                   command=["python"],
                                   args=[
                                       "-m",
                                       "alibiexplainer",
                                       "--model_name","%s"%MODEL_NAME,
                                       "--predictor_host","{0}-{1}.{2}".format(MODEL_NAME, "predictor-default", namespace),
                                       "AnchorTabular"
                                   ]))))

isvc = V1alpha2InferenceService(api_version=api_version,
                          kind=constants.KFSERVING_KIND,
                          metadata=k8s_client.V1ObjectMeta(
                              name=MODEL_NAME, namespace=namespace),
                                spec=V1alpha2InferenceServiceSpec(default=default_endpoint_spec)
                                )

## Create the InferenceService

In [12]:
KFServing = KFServingClient()
KFServing.create(isvc)

{'apiVersion': 'serving.kubeflow.org/v1alpha2',
 'kind': 'InferenceService',
 'metadata': {'creationTimestamp': '2020-07-08T11:33:30Z',
  'generation': 1,
  'name': 'blerssi-model',
  'namespace': 'anonymous',
  'resourceVersion': '92812469',
  'selfLink': '/apis/serving.kubeflow.org/v1alpha2/namespaces/anonymous/inferenceservices/blerssi-model',
  'uid': 'd9346f81-c10e-11ea-90db-42010aa0004e'},
 'spec': {'default': {'explainer': {'custom': {'container': {'args': ['-m',
       'alibiexplainer',
       '--model_name',
       'blerssi-model',
       '--predictor_host',
       'blerssi-model-predictor-default.anonymous',
       'AnchorTabular'],
      'command': ['python'],
      'env': [{'name': 'STORAGE_URI',
        'value': 'pvc://workspace-prometheus/explainer'}],
      'image': 'gcr.io/kfserving/alibi-explainer@sha256:31d487ec6ec9abc6f851c4e481d75a77aea1d8e79d39f18b07bbc8684708fd3e',
      'name': 'kfserving-container',
      'resources': {'limits': {'cpu': '1', 'memory': '2Gi'},
  

## Check the InferenceService

In [13]:
KFServing = KFServingClient()
KFServing.get(MODEL_NAME, namespace=namespace, watch=True)

NAME                 READY      DEFAULT_TRAFFIC CANARY_TRAFFIC  URL                                               
blerssi-model        False                                                                                        
blerssi-model        False                                                                                        
blerssi-model        False                                                                                        
blerssi-model        False                                                                                        
blerssi-model        True       100                             http://blerssi-model.anonymous.example.com/v1/m...


## Run a prediction

In [14]:
CLUSTER='ucs' #where your cluster running 'gcp' or 'ucs'

In [15]:
%%bash -s "$CLUSTER" --out NODE_IP
if [ $1 = "ucs" ]
then
    echo "$(kubectl get node -o=jsonpath='{.items[0].status.addresses[0].address}')"
else
    echo "$(kubectl get node -o=jsonpath='{.items[0].status.addresses[1].address}')"
fi

In [16]:
%%bash --out INGRESS_PORT
INGRESS_GATEWAY="istio-ingressgateway"
echo "$(kubectl -n istio-system get service $INGRESS_GATEWAY -o jsonpath='{.spec.ports[1].nodePort}')"

In [17]:
%%bash -s "$MODEL_NAME" --out SERVICE_HOSTNAME
echo "$(kubectl get inferenceservice $1 -o jsonpath='{.status.url}' | cut -d "/" -f 3)"

### Data for prediction

In [18]:
path="cisco-kubeflow-starter-pack/apps/networking/ble-localization/onprem"
df_full = pd.read_csv(os.path.join(path,'data/iBeacon_RSSI_Unlabeled_truncated.csv')) #Labeled dataset
  # Input Data Preprocessing 
df_full = df_full.drop(['date'],axis = 1)
df_full = df_full.drop(['location'],axis = 1)
df_full[FEATURES] = (df_full[FEATURES])/(-200)
input_data=df_full.to_numpy()[:1]

In [19]:
headers = {"Host": SERVICE_HOSTNAME.strip()}
if not len(input_data)==13 and not type(input_data[0]) ==list:
    data={"instances": input_data.tolist()}
else:
    data={"instances":input_data.tolist()}
url = f"http://{NODE_IP.strip()}:{INGRESS_PORT.strip()}/v1/models/{MODEL_NAME}:predict"
response=requests.post(url, data=json.dumps(data), headers=headers)
print(response.json())

{'predictions': [54]}


## Prediction of the model and explain

In [21]:
if not len(input_data)==13 and not type(input_data[0]) ==list:
    data={"instances": input_data.tolist()}
else:
    data={"instances":[input_data.tolist()]}
url = f"http://{NODE_IP.strip()}:{INGRESS_PORT.strip()}/v1/models/{MODEL_NAME}:explain"
response=requests.post(url, data=json.dumps(data), headers=headers)
print(response.json())

{'meta': {'name': 'AnchorTabular', 'type': ['blackbox'], 'explanations': ['local'], 'params': {'seed': None, 'disc_perc': [25, 50, 75], 'threshold': 0.95, 'delta': 0.1, 'tau': 0.15, 'batch_size': 100, 'coverage_samples': 10000, 'beam_size': 1, 'stop_on_first': False, 'max_anchor_size': None, 'min_samples_start': 100, 'n_covered_ex': 10, 'binary_cache_size': 10000, 'cache_margin': 1000, 'verbose': False, 'verbose_every': 1, 'kwargs': {}}}, 'data': {'anchor': ['0.39 < b3002 <= 1.00', '0.40 < b3004 <= 1.00', 'b3009 <= 1.00', 'b3012 <= 1.00', 'b3011 <= 1.00', 'b3013 <= 1.00', 'b3006 <= 1.00', 'b3003 <= 1.00', 'b3010 <= 1.00', 'b3005 <= 1.00', 'b3001 <= 1.00', 'b3007 <= 1.00', 'b3008 <= 1.00'], 'precision': 0.04072883172561629, 'coverage': 1.0, 'raw': {'feature': [1, 3, 8, 11, 10, 12, 3, 5, 2, 1, 9, 4, 0, 6, 7], 'mean': [0.0297339593114241, 0.04072883172561629, 0.04072883172561629, 0.04072883172561629, 0.04072883172561629, 0.04072883172561629, 0.04072883172561629, 0.04072883172561629, 0.040

## Clean Up

### Delete the InferenceService

In [22]:
KFServing.delete(MODEL_NAME, namespace=namespace)

{'kind': 'Status',
 'apiVersion': 'v1',
 'metadata': {},
 'status': 'Success',
 'details': {'name': 'blerssi-model',
  'group': 'serving.kubeflow.org',
  'kind': 'inferenceservices',
  'uid': 'd9346f81-c10e-11ea-90db-42010aa0004e'}}

In [23]:
!rm -rf $TF_MODEL_DIR
!rm -rf $EXPLAINER_PATH